In [ ]:
import LoopSequencer
import fh2
fh = fh2.FH2()
l = LoopSequencer.LoopSequencer(fh)

In [ ]:
l.setBankTransform(1, "rev")

In [ ]:
l.shiftGrid("up", 1)

In [ ]:
l.clearTransformations(1)

In [ ]:
# 1 INDEXED !!!
l.transformLoop(1, "rev", send=True)
# l.transformLoop(1, "transpose", direction="up", interval="m2", send=True)
# l.transformLoop(7, "invert", send=False)

In [ ]:
l.replaceRow(1, [1, 0,1,0,1,1,1,1])

In [ ]:
n += 1
l.paramValues['TEMPO']
l.setParam('TEMPO', n)

In [ ]:
n -= 2
l.paramValues['TEMPO']
l.setParam('TEMPO', n)

In [ ]:
nl = l.transformLoop(2, "rev", send=False)

nl2 = l.transformLoop(2, "transpose", direction="down", interval="m7", send=False)
send = l.transformLoop(7, "invert", send=False)
l.transformLoop(3, "transpose", direction="up", interval="m3", send=False)
l.transformLoop(3, "rev", send=False)


In [ ]:
fh.saveScenesToFile('controllerize1_e.scenes')

In [ ]:
# #LOAD - 

fh.loadScenesFromFile('controllerize1_e.scenes')

In [ ]:
fh.superColliderServer.addMsgHandler("/utilityButton", lambda a, b, c, d : None)

In [1]:
#sustainPad -> arpeggiator setup
import PydalChanel as pydalmodule
import arpeggiator
from spatializer import Spatializer
import LoopSequencer
import fh2

fh = fh2.FH2()
l = LoopSequencer.LoopSequencer(fh)
arpPydal = pydalmodule.getPydalInstance(('127.0.0.1', 34567))
read = pydalmodule.read
lp = fh

instChan = 1
s = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, instChan)
s.spatialize = False
s.separateChannels = False
s.broadcasting = True
lp.addForwardingHandler(instChan, s)
# s.loadChordsFromFile("songV4.chords")

a = arpeggiator.Arpeggiator(instChan, arpPydal, read("0 1 2", 1, "arp"))
a.normalForwardingBehavior = False
lp.addForwardingHandler(instChan, a)

In [ ]:
instChan = 3
s1 = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, instChan)
s1.spatialize = False
s1.separateChannels = False
s1.broadcasting = True
lp.addForwardingHandler(instChan, s1)
# s.loadChordsFromFile("songV4.chords")

a1 = arpeggiator.Arpeggiator(instChan, arpPydal, read("0 1 2", 1, "arp"))
a1.normalForwardingBehavior = False
lp.addForwardingHandler(instChan, a1)
# lp.resetButtonDestinations(["-1"]*40 + ["-0"]*40)

In [ ]:
instChan = 5
a5 = arpeggiator.Arpeggiator(instChan, arpPydal, read("0 1 2", 1, "arp"))
lp.addForwardingHandler(instChan, a5)

In [ ]:
lp.resetButtonDestinations(["-1"]*80)
def toggleSustain():
    s1.sustaining = not s1.sustaining
def toggleSeparateChannels():
    s1.separateChannels = not s1.separateChannels
lp.topRowFunctions[0] = toggleSustain
lp.topRowFunctions[1] = toggleSeparateChannels
s1.loadChordsFromFile("droneSong3.chords")

In [ ]:
b = True
s.broadcasting = b
s1.broadcasting = b

In [ ]:
s.loadChordsFromFile("controllerize1_1.chords")
s1.loadChordsFromFile("controllerize1_2.chords")

In [ ]:
s.saveChordsToFile("controllerize1_1.chords")
s1.saveChordsToFile("controllerize1_2.chords")

In [ ]:
s.swappingChords = True
s1.swappingChords = True

In [2]:
pydalInstance = pydalmodule.getPydalInstance(('127.0.0.1', 34345))
ch1 = pydalInstance.newChannel(1)
ch2 = pydalInstance.newChannel(2)

In [30]:
ch1.play(read("{~ ~ ~ ~, b ~ ~ ~ ~ ~ ~ b ~ ~ ~ ~ ~ b ~ ~ ~ ~ b ~ ~ ~ b ~ ~ b ~ b}", 1, 'roboDrum'))

In [31]:
ch2.play(read("{~ ~ ~ ~, a ~ ~ ~}", 1, 'roboDrum'))

In [32]:
ch2.stop()
ch1.stop()
ch1.play(read("{~ ~ ~ ~, a ~ ~ a ~ ~ a ~ b ~ ~ ~ a ~ ~ a ~ ~ b ~ a ~ ~ a ~ ~ b ~ }", 1, 'roboDrum'))

In [33]:
ch1.play(read("{~ ~ ~ ~, a ~ b a ~ b a ~ b b ~ ~ a ~ b a ~ b b ~ a ~ b a ~ b b ~ }", 1, 'roboDrum'))

In [34]:
ch1.play(read("{~ ~ ~ ~, a ~ ~ a ~ ~ a ~ a ~ ~ ~ a ~ ~ a ~ ~ a ~ a ~ ~ a ~ ~ a ~ }", 1, 'roboDrum'))
ch2.play(read("{~ ~ ~ ~, b ~ ~ ~ ~ ~ ~ ~}", 1, 'roboDrum'))

In [35]:
ch2.stop()
ch1.stop()